                                                                                    ETL - STEAM

- Librerias a usar

In [1]:
import pandas as pd
import numpy as np
import ast
import json
from datetime import datetime
from collections import Counter
import re

- Leemos el archivo output_steam_games

In [2]:
rows = []
with open('output_steam_games.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(json.loads(line))

df_output = pd.DataFrame(rows)

In [3]:
df_output

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


- Eliminar las filas en las que todos los valores son nulos pasamos de 120.445 filas a 32135 filas

In [4]:

df_output.dropna(axis=0, how='all', inplace=True)

- Eliminamos la columna 'app_name' dado que es la misma que title
- Eliminamos la columna reviews_url dado que ya tenemos información en el df de reviews

In [5]:

df_output.drop(columns=['app_name', 'reviews_url'], inplace= True)

- Reemplazamos el nombre de la columna 'id' por 'item_id'

In [6]:
df_output.rename(columns={'id': 'item_id'}, inplace=True)

- Eliminamos la columna publisher dado que no aporta información relevante para el análisis

In [7]:
df_output.drop(columns='publisher', inplace= True)

Analizando la columna genres queremos ver las diferentes categorias que definene el género de los videojuegos

- Crear una lista para almacenar todas las palabras (categorías)

In [8]:
all_categories = []
# Recorrer cada celda de la columna 'genres'
for genres_list in df_output['genres']:
    if isinstance(genres_list, list):  # Verificar si es una lista antes de iterar
        all_categories.extend(genres_list)

- Calcular la frecuencia de cada palabra (categoría)

In [9]:
word_frequencies = Counter(all_categories)

In [10]:
word_frequencies

Counter({'Action': 11321,
         'Casual': 8282,
         'Indie': 15858,
         'Simulation': 6699,
         'Strategy': 6957,
         'Free to Play': 2031,
         'RPG': 5479,
         'Sports': 1257,
         'Adventure': 8243,
         'Racing': 1083,
         'Early Access': 1462,
         'Massively Multiplayer': 1108,
         'Animation &amp; Modeling': 183,
         'Video Production': 116,
         'Utilities': 340,
         'Web Publishing': 268,
         'Education': 125,
         'Software Training': 105,
         'Design &amp; Illustration': 460,
         'Audio Production': 93,
         'Photo Editing': 77,
         'Accounting': 7})

Las siguientes lineas de código se realizan para revisar la información contenida en la columna genre y tags

- Analizando la columna genres queremos ver las diferentes categorias que definene el género de los videojuegos
- Crear una lista para almacenar todas las palabras (categorías)

In [11]:
all_categories = []
# Recorrer cada celda de la columna 'genres'
for genres_list in df_output['genres']:
    if isinstance(genres_list, list):  # Verificar si es una lista antes de iterar
        all_categories.extend(genres_list)

In [12]:
# Calcular la frecuencia de cada palabra (categoría)
word_frequencies = Counter(all_categories)
# Mostrar las palabras y sus frecuencias
for word, frequency in word_frequencies.items():
    print(f'{word}: {frequency}')

Action: 11321
Casual: 8282
Indie: 15858
Simulation: 6699
Strategy: 6957
Free to Play: 2031
RPG: 5479
Sports: 1257
Adventure: 8243
Racing: 1083
Early Access: 1462
Massively Multiplayer: 1108
Animation &amp; Modeling: 183
Video Production: 116
Utilities: 340
Web Publishing: 268
Education: 125
Software Training: 105
Design &amp; Illustration: 460
Audio Production: 93
Photo Editing: 77
Accounting: 7


- Analizando la columna tags queremos ver si hay similitud con la columna genre
- Crear una lista para almacenar todas las palabras (tags)

In [13]:
all_tags = []
# Recorrer cada celda de la columna 'tags'
for tags_list in df_output['tags']:
    if isinstance(tags_list, list):  # Verificar si es una lista antes de iterar
        all_tags.extend(tags_list)
# Calcular la frecuencia de cada palabra (tag)
tag_frequencies = Counter(all_tags)
# Mostrar las palabras y sus frecuencias
for tag, frequency in tag_frequencies.items():
    print(f'{tag}: {frequency}')

Strategy: 7566
Action: 13005
Indie: 17620
Casual: 9739
Simulation: 7730
Free to Play: 2406
RPG: 5919
Card Game: 233
Trading Card Game: 76
Turn-Based: 949
Fantasy: 1363
Tactical: 504
Dark Fantasy: 170
Board Game: 210
PvP: 236
2D: 1945
Competitive: 254
Replay Value: 391
Character Customization: 194
Female Protagonist: 1178
Difficult: 1313
Design & Illustration: 629
Sports: 1597
Multiplayer: 2383
Adventure: 9831
FPS: 1009
Shooter: 1234
Third-Person Shooter: 234
Sniper: 22
Third Person: 622
Racing: 1229
Early Access: 1947
Survival: 864
Pixel Graphics: 1241
Cute: 558
Physics: 457
Science: 69
VR: 1796
Tutorial: 98
Classic: 794
Gore: 643
1990's: 145
Singleplayer: 4346
Sci-fi: 1388
Aliens: 163
First-Person: 1086
Story Rich: 1446
Atmospheric: 1935
Silent Protagonist: 36
Great Soundtrack: 2235
Moddable: 245
Linear: 63
Retro: 959
Funny: 1117
Turn-Based Strategy: 444
Platformer: 1449
Side Scroller: 351
Massively Multiplayer: 1286
Clicker: 92
Gothic: 43
Isometric: 229
Stealth: 348
Mystery: 448
Assa

Eliminamos la columa tags dado que ofrece información muy específica para cada juego y orientada al género 
lo cual es redundante para el estudio.

In [14]:
df_output.drop(columns='tags', inplace= True)
# Crear un conjunto de todas las categorías únicas
all_categories = set()
for genres_list in df_output['genres']:
    if isinstance(genres_list, list):
        all_categories.update(genres_list)
# Generar las columnas codificadas para cada categoría
for category in all_categories:
    df_output[category] = df_output['genres'].apply(lambda x: 1 if isinstance(x, list) and category in x else 0)

Las siguientes lineas de código se realizan para revisar la información contenida en la columna specs

In [15]:
# Crear una lista para almacenar todas las palabras (specs)
all_specs = []
# Recorrer cada celda de la columna 'tags'
for specs_list in df_output['specs']:
    if isinstance(specs_list, list):  # Verificar si es una lista antes de iterar
        all_specs.extend(specs_list)
# Calcular la frecuencia de cada palabra (specs)
specs_frequencies = Counter(all_specs)
# Mostrar las palabras y sus frecuencias
for specs, frequency in specs_frequencies.items():
    print(f'{specs}: {frequency}')
    

Single-player: 27577
Multi-player: 8198
Online Multi-Player: 2370
Cross-Platform Multiplayer: 2550
Steam Achievements: 16366
Steam Trading Cards: 12199
In-App Purchases: 933
Stats: 2321
Full controller support: 7385
HTC Vive: 1984
Oculus Rift: 1213
Tracked Motion Controllers: 1816
Room-Scale: 1445
Downloadable Content: 12691
Steam Cloud: 9960
Steam Leaderboards: 6081
Partial Controller Support: 6539
Seated: 1090
Standing: 1462
Local Co-op: 743
Shared/Split Screen: 3962
Valve Anti-Cheat enabled: 462
Local Multi-Player: 1237
Steam Turn Notifications: 91
MMO: 942
Co-op: 4280
Online Co-op: 998
Captions available: 1643
Commentary available: 201
Steam Workshop: 1866
Includes level editor: 1377
Mods: 11
Mods (require HL2): 10
Game demo: 363
Includes Source SDK: 60
SteamVR Collectibles: 30
Keyboard / Mouse: 418
Gamepad: 387
Windows Mixed Reality: 161
Mods (require HL1): 1


- Revisamos la cantidad de categorías que tenemos en specs
- Crear un conjunto de todas las especificaciones únicas

In [16]:
unique_specs = set()
for specs_list in df_output['specs']:
    if isinstance(specs_list, list):
        unique_specs.update(specs_list)

- Obtener la cantidad de especificaciones únicas

In [17]:
num_unique_specs = len(unique_specs)

In [18]:
print(f'Cantidad de especificaciones únicas: {num_unique_specs}')

Cantidad de especificaciones únicas: 40


- Eliminamos la columa specs dado que tiene 40 categorías que pueden ser relacionadas también al genero del videojuego

In [19]:
df_output.drop(columns=['genres', 'specs'], inplace= True)

- Filtrar y mostrar los valores únicos en la columna 'price' que contengan letras

In [20]:
unique_prices_with_letters = df_output['price'].loc[df_output['price'].apply(lambda x: bool(re.search('[a-zA-Z]', str(x))))].unique()
print(unique_prices_with_letters)

['Free To Play' 'Free to Play' nan 'Free' 'Free Demo' 'Play for Free!'
 'Install Now' 'Play WARMACHINE: Tactics Demo' 'Free Mod' 'Install Theme'
 'Third-party' 'Play Now' 'Free HITMAN™ Holiday Pack' 'Play the Demo'
 'Starting at $499.00' 'Starting at $449.00' 'Free to Try' 'Free Movie'
 'Free to Use']


- Reemplazamos 'Free To Play' por 0 en la columna 'price'


In [21]:
df_output['price'] = df_output['price'].replace('Starting at $499.00', 499)
df_output['price'] = df_output['price'].replace('Starting at $449.00', 449)

- Valores a reemplazar por ceros


In [22]:
values_to_replace = ['Free HITMAN™ Holiday Pack', 'Play Now', 'Third-party','Free To Play', 'Free Mod', 'Install Theme', 
                     'Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 
                     'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use']

- Reemplazar los valores por ceros en la columna 'price'


In [23]:
df_output['price'] = df_output['price'].replace(values_to_replace, 0)

- Convertir la columna 'price' a tipo de dato float


In [24]:
df_output['price'] = df_output['price'].astype(float)

- Supongamos que queremos reemplazar los valores NaN con False (0)


In [25]:
df_output['early_access'].fillna(False, inplace=True)


- Ahora, podemos convertir la columna 'early_access' en valores enteros (binarios)


In [26]:
df_output['early_access'] = df_output['early_access'].astype(int)


- Filtrar y mostrar los valores únicos en 'release_date' que contienen letras (excluyendo nulos)


In [27]:
df_output['release_date'].loc[df_output['release_date'].apply(lambda x: isinstance(x, str) and any(c.isalpha() for c in x))].unique()

array(['Soon..', 'Beta测试已开启', 'Jun 2009', 'Oct 2010', 'Feb 2011',
       'Aug 2014', 'Sep 2014', 'Apr 2015', 'Apr 2016', 'Jul 2016',
       'June 2016', 'Coming Soon', 'Q2 2017', 'TBA', "When it's done",
       'coming soon', 'Q2 2018', 'Winter 2017', 'soon', 'Mar 2018',
       '14 July', 'Jul 2017', 'Summer 2017', 'Spring 2018', 'Winter 2018',
       'To be Announced', 'October 2017', 'TBD', 'Fall 2017', 'Nov 2017',
       'Q1 2018', 'Dec 2017', 'Январь 2018', 'Soon',
       'First quarter of 2018', 'H2 2018', '21 Jun, 2017', '2018年初頭発売予定',
       'Please wait warmly', 'early access', 'SOON', 'Feb 2018',
       'Coming Soon/Próximamente', 'August 2017',
       '2018 [Now get free Pre Alpha]', '1st Quarter 2018', 'Jan 2018',
       '预热群52756441', '2018年1月', 'Coming soon', 'Fall 2018', '"""Soon"""',
       'Spring 2017', 'Demo coming soon.', 'Coming 2017',
       'Not yet available',
       "C'est bientôt...                    (ou pas)", 'January 2019',
       'Q4 2017', 'Early 2018', '


Al revisar la data vemos que tenemos muchos valores en diferentes formatos por lo que se asume que vamos a tomar solo 
el año de lanzamiento del videojuego dado que la consulta developer solo pide el año.

- Filtrar y extraer solo los años numéricos de la columna 'release_date'


In [28]:
df_output['release_year'] = df_output['release_date'].apply(lambda x: re.findall(r'\d{4}', str(x))[0] if isinstance(x, str) and re.match(r'\d{4}', str(x)) else None)

- Eliminamos la columna release_date y nos quedamos con release_year


In [29]:
df_output.drop(columns='release_date', inplace=True)

- Eliminamos las dos finas que tenemos vacías en item_id


In [30]:
df_output = df_output.dropna(subset=['item_id'])

                                                        Transformar el arhcivo en un CSV 

In [36]:
df_output.to_csv('steam.csv', index=False)

In [37]:
df= pd.read_csv('steam.csv')

In [38]:
df

,title,url,price,early_access,item_id,developer,Web Publishing,Indie,Education,Animation &amp; Modeling,...,Audio Production,Adventure,Racing,Simulation,Sports,Strategy,Action,Design &amp; Illustration,Free to Play,release_year
0,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,4.99,0,761140,Kotoshiro,0,1,0,0,...,0,0,0,1,0,1,1,0,0,2018.0
1,Ironbound,http://store.steampowered.com/app/643980/Ironb...,0.00,0,643980,Secret Level SRL,0,1,0,0,...,0,0,0,0,0,1,0,0,1,2018.0
2,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,0.00,0,670290,Poolians.com,0,1,0,0,...,0,0,0,1,1,0,0,0,1,2017.0
3,弹炸人2222,http://store.steampowered.com/app/767400/2222/,0.99,0,767400,彼岸领域,0,0,0,0,...,0,1,0,0,0,0,1,0,0,2017.0
4,NaN,http://store.steampowered.com/app/773570/Log_C...,2.99,0,773570,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,1.99,0,773640,"Nikita ""Ghost_RUS""",0,1,0,0,...,0,0,0,1,0,1,0,0,0,2018.0
32129,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,4.99,0,733530,Sacada,0,1,0,0,...,0,0,0,0,0,1,0,0,0,2018.0
32130,Russian Roads,http://store.steampowered.com/app/610660/Russi...,1.99,0,610660,Laush Dmitriy Sergeevich,0,1,0,0,...,0,0,1,1,0,0,0,0,0,2018.0
32131,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,4.99,0,658870,"xropi,stev3ns",0,1,0,0,...,0,0,0,0,0,0,0,0,0,2017.0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 29 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   title                      30084 non-null  object 
 1   url                        32133 non-null  object 
 2   price                      30756 non-null  float64
 3   early_access               32133 non-null  int64  
 4   item_id                    32133 non-null  int64  
 5   developer                  28835 non-null  object 
 6   Web Publishing             32133 non-null  int64  
 7   Indie                      32133 non-null  int64  
 8   Education                  32133 non-null  int64  
 9   Animation &amp; Modeling   32133 non-null  int64  
 10  Early Access               32133 non-null  int64  
 11  Photo Editing              32133 non-null  int64  
 12  Utilities                  32133 non-null  int64  
 13  Massively Multiplayer      32133 non-null  int

El siguiente archivo csv se crea para facilitar la consulta en la API

- Importamos el archivo items recientemente creado


In [39]:
expanded_items_df = pd.read_parquet('items1.parquet')
expanded_items_df['item_id'] = expanded_items_df['item_id'].astype(str)

- Filtramos los df para hacer mejor la busqueda 


In [40]:
df_intems_filtered = expanded_items_df[['item_id', 'playtime_forever']]

- Eliminamos columnas que no necesitamos


In [41]:
columas_para_excluir = ['title', 'url', 'price', 'early_access', 'developer', 'release_year']
new_df_games = df_output.drop(columns=columas_para_excluir)

- Hacemos un merge del df games con el df filtrado


In [42]:
merge_df = pd.merge(df_intems_filtered, new_df_games, on= 'item_id', how= 'left')

- Crear un nuevo dataframe para guardar los resultados


In [43]:
result_df = pd.DataFrame(columns=['Total_Playtime'])

- Realizar el ciclo for para calcular sumatoria por columna


In [44]:
for column in merge_df.columns:
    if column != 'playtime_forever':
        total_playtime = merge_df[merge_df[column] == 1]['playtime_forever'].sum()
        result_df.at[column, 'Total_Playtime'] = total_playtime

- Ordenar el dataframe de mayor a menor


In [45]:
result_df_sorted = result_df.sort_values(by='Total_Playtime', ascending=False)

- Agregar una columna de posición numérica


In [46]:
result_df_sorted.insert(0, 'Position', range(1, len(result_df_sorted) + 1))

In [47]:
result_df_sorted.to_csv('genre.csv')

In [48]:
df= pd.read_csv('genre.csv')

In [44]:
df

,Unnamed: 0,Position,Total_Playtime
0,Action,1,309440800
1,Indie,2,148387992
2,RPG,3,103869655
3,Adventure,4,90956921
4,Simulation,5,86587199
5,Strategy,6,66206390
6,Free to Play,7,61846335
7,Massively Multiplayer,8,44681760
8,Casual,9,24894345
9,Early Access,10,14378281


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      23 non-null     object
 1   Position        23 non-null     int64 
 2   Total_Playtime  23 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 684.0+ bytes
